# Для анализа был выбран dataset по прокату фильмов

**Цель: Ответить на вопросы используя pandas**
- В Docker контейнерах были подняты сервера Jupyter Notebook и PostgreSQL <br>
- База была развернута из *.backup файла 

In [ ]:
!pip install psycopg2

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import sqlalchemy

**Добавляем настройки вывода**

In [3]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 2000)

In [4]:
engine = sqlalchemy.create_engine('postgresql+psycopg2://postgres:postgres@db:5432/db_film')

**Получаем список таблиц базы из схемы "dvd-rental"**

In [13]:
t = pd.read_sql_query(sql="""
    SELECT table_name FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
    AND table_schema IN('dvd-rental');
    """,
    con=engine)

In [16]:
t['table_name'].unique()

array(['actor', 'actor_info', 'author', 'customer_list', 'film_list',
       'nicer_but_slower_film_list', 'orders', 'sales_by_film_category',
       'payment', 'address', 'category', 'city', 'country', 'customer',
       'film_actor', 'film_category', 'inventory', 'language', 'rental',
       'staff', 'sales_by_store', 'staff_list', 'test', 'view_v', 'film',
       'store'], dtype=object)

**Получаем нужные таблицы в формате DataFrame**

In [9]:
film = pd.read_sql_table('film', engine, schema='dvd-rental')
actor = pd.read_sql_table('actor', engine, schema='dvd-rental')
film_actor = pd.read_sql_table('film_actor', engine, schema='dvd-rental')
address = pd.read_sql_table('address', engine, schema='dvd-rental')
payment = pd.read_sql_table('payment', engine, schema='dvd-rental')
customer = pd.read_sql_table('customer', engine, schema='dvd-rental')
city = pd.read_sql_table('city', engine, schema='dvd-rental')
country = pd.read_sql_table('country', engine, schema='dvd-rental')
film_category = pd.read_sql_table('film_category', engine, schema='dvd-rental')
category = pd.read_sql_table('category', engine, schema='dvd-rental')
rental = pd.read_sql_table('rental', engine, schema='dvd-rental')

In [10]:
actor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     200 non-null    int64         
 1   first_name   200 non-null    object        
 2   last_name    200 non-null    object        
 3   last_update  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 6.4+ KB


 ## Вопросы

1) Получить список с уникальными названиями регионов из таблицы адресов, чтобы запрос выводил только те регионы, названия которых начинаются на "K" и заканчиваются на "a", и названия не содержат пробелов

In [12]:
address.loc[
          (address['district'].str.startswith('K')) &
          (address['district'].str.endswith('a')) &
          (~address['district'].str.contains(' '))
      ]['district'].unique()

array(['Kanagawa', 'Kalmykia', 'Kaduna', 'Karnataka', 'Ktahya', 'Kerala',
       'Kitaa'], dtype=object)

2) Получить из таблицы платежей за прокат фильмов информацию по платежам, которые выполнялись в промежуток с 17 марта 2007 года по 19 марта 2007 года включительно, и стоимость которых превышает 1.00

In [11]:
payment.loc[
          (payment['payment_date'] >= '2007-03-17') & (payment['payment_date'] < '2007-03-20') &
          (payment['amount'] >= 1)
          ].sort_values('payment_date', ascending=False)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date
7564,25067,258,1,13491,4.99,2007-03-19 23:59:22.996577
3677,21180,443,1,13489,2.99,2007-03-19 23:57:32.996577
3831,21334,458,2,13487,2.99,2007-03-19 23:55:31.996577
3659,21162,441,2,13483,2.99,2007-03-19 23:45:04.996577
6946,24449,183,1,13482,2.99,2007-03-19 23:42:56.996577
...,...,...,...,...,...,...
2267,19770,293,1,11576,2.99,2007-03-17 00:21:46.996577
6565,24068,146,1,11573,4.99,2007-03-17 00:06:44.996577
4367,21870,515,2,11572,4.99,2007-03-17 00:06:21.996577
2296,19799,296,2,11571,4.99,2007-03-17 00:06:17.996577


3) Выведите информацию о 10-ти последних платежах за прокат фильмов.

In [17]:
payment[['payment_id', 'customer_id', 'amount', 'payment_date']].sort_values(by='payment_date', ascending=False).head(10).reset_index(drop=True)

,payment_id,customer_id,amount,payment_date
0,32098,264,2.99,2007-05-14 13:44:29.996577
1,31983,516,0.00,2007-05-14 13:44:29.996577
2,31981,512,0.99,2007-05-14 13:44:29.996577
3,31980,508,0.99,2007-05-14 13:44:29.996577
4,31979,505,4.99,2007-05-14 13:44:29.996577
5,31978,497,4.99,2007-05-14 13:44:29.996577
6,31977,496,2.99,2007-05-14 13:44:29.996577
7,31976,495,0.99,2007-05-14 13:44:29.996577
8,31975,493,2.99,2007-05-14 13:44:29.996577
9,31974,479,0.99,2007-05-14 13:44:29.996577


4) Выведите следующую информацию по покупателям: <br>
-- Фамилия и имя (в одной колонке через пробел) <br>
-- Электронная почта <br>
-- Длину значения поля email

In [22]:
customer['full_name'] = customer['last_name'].str.cat(customer['first_name'], sep=' ')
customer['len_email'] = customer['email'].str.len()
customer[['full_name','email', 'len_email']]

,full_name,email,len_email
0,Ely Jared,jared.ely@sakilacustomer.org,28
1,Smith Mary,mary.smith@sakilacustomer.org,29
2,Johnson Patricia,patricia.johnson@sakilacustomer.org,35
3,Williams Linda,linda.williams@sakilacustomer.org,33
4,Jones Barbara,barbara.jones@sakilacustomer.org,32
...,...,...,...
594,Gunderson Terrence,terrence.gunderson@sakilacustomer.org,37
595,Forsythe Enrique,enrique.forsythe@sakilacustomer.org,35
596,Duggan Freddie,freddie.duggan@sakilacustomer.org,33
597,Delvalle Wade,wade.delvalle@sakilacustomer.org,32


5)Выведите для каждого покупателя его адрес проживания, город и страну.

In [24]:
customer.merge(address, how='left', left_on='address_id', right_on='address_id')\
      .merge(city, how='left', on='city_id')\
      .merge(country, how='left', on='country_id', suffixes=('_left', '_right'))\
          [['customer_id', 'address', 'city', 'country']]

,customer_id,address,city,country
0,524,1003 Qinhuangdao Street,Purwakarta,Indonesia
1,1,1913 Hanoi Way,Sasebo,Japan
2,2,1121 Loja Avenue,San Bernardino,United States
3,3,692 Joliet Street,Athenai,Greece
4,4,1566 Inegl Manor,Myingyan,Myanmar
...,...,...,...,...
594,595,844 Bucuresti Place,Jinzhou,China
595,596,1101 Bucuresti Boulevard,Patras,Greece
596,597,1103 Quilmes Boulevard,Sullana,Peru
597,598,1331 Usak Boulevard,Lausanne,Switzerland


6) Посчитайте для каждого магазина количество его покупателей.

In [26]:
store_customer = customer.groupby('store_id').count()['customer_id'].reset_index()
store_customer

,store_id,customer_id
0,1,326
1,2,273


6.1) Доработайте запрос и выведите только те магазины, у которых количество покупателей больше 300

In [27]:
store_customer[store_customer['customer_id'] > 300]

,store_id,customer_id
0,1,326


7) Посчитайте для каждого покупателя:<br>
-- Минимальный платеж <br>
-- Максимальный платеж <br>
-- Сумму по всем платежам <br>
-- Количество платежей

In [28]:
payment.groupby('customer_id').agg({'amount': [min, max, sum], 'payment_id': 'count'}).reset_index()

customer_id amount                payment_id
                   min    max     sum      count
0             1   0.99   9.99  114.70         30
1             2   0.99  10.99  123.74         26
2             3   0.99  10.99  130.76         24
3             4   0.99   8.99   81.78         22
4             5   0.99   9.99  134.65         35
..          ...    ...    ...     ...        ...
594         595   0.99  10.99  110.71         29
595         596   0.99   6.99   73.78         22
596         597   0.99   6.99   87.77         23
597         598   0.99   7.99   83.78         22
598         599   0.99   9.99   78.82         18

[599 rows x 5 columns]

8) Добавьте столбец с классификацией длины фильма и посчитайте количество фильмов в каждой категории <br>
-- Длинна < 60 - Короткий <br>
-- Длинна >= 60 и длинна < 130 - Средний <br>
-- Длинна > 130 - Длинный

In [37]:
def film_class(lenth: int):
    if lenth < 60 and lenth >= 1:
        return 'Short'
    elif lenth >= 60 and lenth < 130:
        return 'Average'
    elif lenth > 130:
        return 'Long'

film['lenth_class'] = film['length'].apply(film_class)
film[['film_id', 'length', 'lenth_class']]

,film_id,length,lenth_class
0,133,117,Average
1,384,49,Short
2,8,54,Short
3,98,73,Average
4,1,86,Average
...,...,...,...
995,996,183,Long
996,997,179,Long
997,998,105,Average
998,999,101,Average


In [34]:
film['lenth_class'].value_counts()

Average    512
Long       386
Short       96
Name: lenth_class, dtype: int64

9) Найдите имена всех актеров снимавшихся в фильме 'Alabama Devil' и 'Airport Pollock'

In [38]:
actors_names = film_actor.merge(actor, how='left', on='actor_id').merge(film, how='left', on='film_id')[['film_id', 'actor_id', 'first_name', 'last_name', 'title']]
actors_names.loc[
          (actors_names['title'].str.contains('Alabama Devil|Airport Pollock'))
      ].sort_values('title')

,film_id,actor_id,first_name,last_name,title
1433,8,55,Fay,Kilmer,Airport Pollock
2559,8,96,Gene,Willis,Airport Pollock
2988,8,110,Susan,Davis,Airport Pollock
3748,8,138,Lucille,Dee,Airport Pollock
213,9,10,Christian,Gable,Alabama Devil
546,9,22,Elvis,Marx,Alabama Devil
654,9,26,Rip,Crawford,Alabama Devil
1379,9,53,Mena,Temple,Alabama Devil
1796,9,68,Rip,Winslet,Alabama Devil
2934,9,108,Warren,Nolte,Alabama Devil


10) Определите количество фильмов в каждой категории по рейтингам

In [44]:
films_in_categories = film_category.merge(film, how='left', on='film_id').merge(category, how='left', on='category_id')[['film_id', 'category_id', 'rating', 'name']]
print(films_in_categories.pivot_table(index=['name'], columns='rating', values='film_id', aggfunc='count', margins=True).reset_index())

rating         name    G  NC-17   PG  PG-13    R   All
0            Action   18     12    9     11   14    64
1         Animation   13     15   11     19    8    66
2          Children   10     12   15     14    9    60
3          Classics   11      9   10     13   14    57
4            Comedy   11     11   16     12    8    58
5       Documentary   14     13   16     12   13    68
6             Drama   12     15    7     22    6    62
7            Family   10     14   18     11   16    69
8           Foreign   13     12   14     19   15    73
9             Games   11     16    5     14   15    61
10           Horror    9      7   12     14   14    56
11            Music    2     20   10      8   11    51
12              New   12     18    9     15    9    63
13           Sci-Fi   10      9   12     13   17    61
14           Sports   11     17   16     14   16    74
15           Travel   11     10   14     12   10    57
16              All  178    210  194    223  195  1000
